# Машинное обучение

## Факультет математики НИУ ВШЭ

### 2018-2019 учебный год

Лектор: Илья Щуров

Семинаристы: Евгения Ческидова, Евгений Ковалев

Ассистенты: Константин Ваниев, Софья Дымченко

# План на сегодня

- вокруг да около линейной регрессии
- feature selection и регуляризация: теория и практика!

## Отбор признаков и регуляризация
На лекции мы обсуждали, что в задаче линейной регрессии (и, на самом деле, не только в ней) иногда лучше отбросить какие-то признаки (то есть принудительно установить соответствующую компоненту вектора весов в 0), чтобы уменьшить разброс вектора весов (и получающихся предсказаний) — даже ценой появления смещённости. Это называется «отбор признаков» (feature selection).

Сейчас мы поговорим о другом методе уменьшения разброса: регуляризации.

### Задача 1
Пусть, как обычно, $X$ — матрица объект-признак (которую мы считаем фиксированной), $w$ — фиксированный вектор весов (истинный), $y=Xw+\varepsilon$ — вектор ответов, $\varepsilon$ — вектор случайных ошибок, $\mathbb E[\varepsilon]=0$, ковариацонная матрица $\varepsilon$ равна $\sigma^2 I$, где $I$ — тождественная матрица (то есть все компоненты вектора $\varepsilon$ имеют одинаковую дисперсию $\sigma^2$, и любая пара разных компонент имеют нулевую ковариацию).

Рассмотрим следующиую оптимизационную задачу:

$$\| X\widehat w - y \| + \lambda \| \widehat w \|^2 \to \min_{\widehat w}, \quad \lambda > 0.$$

Слагаемое $\lambda \| \widehat w \|^2$ называется *регуляризатором*. Он «штрафует» большие значения компонент $w$.

1. Найти её решение в явном виде. 
2. Что будет, если $\lambda$ очень большой? Очень маленький?
2. Если столбцы матрицы $X$ оказались линейно зависимы, однозначно ли определяется $\widehat w$?
2. Является ли полученная таким образом оценка $\widehat w$ несмещённой оценкой для $w$?

**Замечание.** Регрессия, заданная таким образом, называется *гребнёвой* (ridge) регрессией.

### Задача 2
Пусть в условиях задачи 1 матрица $X$ имеет вид:

$$X=\begin{pmatrix}
10 & 0\\
-10 & 0\\
0 & 1\\
0 & -1
\end{pmatrix}.$$

Какая из компонент вектора $w$ сильнее уменьшится в результате добавления регуляризатора? Как вы можете это объяснить? Сравните с обсуждением отбора признаков в этом сюжете.

# Practice

Мы поработаем с данными о сообществах в США. Описание датасета:

http://archive.ics.uci.edu/ml/datasets/communities+and+crime

Датасет на кэггле (в формате .csv):

https://www.kaggle.com/kkanda/communities%20and%20crime%20unnormalized%20data%20set

Будем предсказывать количество насильственных преступлений относительно численности населения.

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
data = pd.read_csv('crimedata.csv', na_values=["?"])
# будем работать не со всеми колонками
requiredColumns = [5, 6] + list(range(11,26)) + list(range(32, 103)) + [145]
data = data[data.columns[requiredColumns]]
# некоторые значения целевой переменной пропущены
X = data.loc[data['ViolentCrimesPerPop'].notnull(), :].drop('ViolentCrimesPerPop', axis=1)
y = data['ViolentCrimesPerPop'][X.index]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### 1 Baseline

Обучите линейную регрессию и выведите качество по метрикам $R^2$ и MSE на обучающей и тестовой выборке.

In [ ]:
# your code here

Добавьте регуляризатор и посмотрите, изменилось ли качество. Попробуйте методы Lasso ($L_1$) и Ridge ($L_2$). Попробуйте также KNN.

**Комментарий.** $L_1$-регуляризатор — это просто $L_1$-норма вектора весов, то есть сумма модулей всех компонент.

In [ ]:
# your code here

Попробуйте поизменять параметр регуляризации ($\lambda$) и посмотреть, что получится. Что будет с происходить с коэффициентами регрессии при увеличении параметра регуляризации?

In [ ]:
# your code here

### 2 Scaling

А что изменится при нормировании признаков? Попробуйте StandardScaler и MinMaxScaler. Есть ли разница? Влияет ли нормирование на предсказания линейной регрессии? А на предсказания регуляризованной? Почему так?

In [ ]:
# your code here

### 3 High/low variance

Полезны ли признаки, имеющие высокую дисперсию? А низкую? Попробуйте удалить их и посмотреть, как изменится качество.

In [ ]:
features_variance = X_train.var().sort_values(ascending=False)

In [ ]:
# your code here

### 4 Correlation

Попробуйте теперь избавиться от самых коррелирующих признаков и посмотрите на изменение качества.

In [ ]:
correlated_features = X.corr(method='pearson').abs().unstack().sort_values(ascending=False)

In [ ]:
# your code here

### 5 Сombination!

Скомбинируйте нормализацию с выбором признаков и посмотрите, изменилось ли качество.

In [ ]:
# your code here

### 6 Hyperparameter tuning

Какой метод обработки данных сработал лучше всех? Используя его, подберите параметр регуляризации и посмотрите на финальное качество!

In [ ]:
# your code here